<a href="https://colab.research.google.com/github/BhaskarBiswas/GoogleColabNotebooks/blob/main/MadAboutSports_Bonus_6_Get_Statsbomb_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get that Data!

Scrape open-data via the Statsbomb Github repo

## Import libraries

In [ ]:
# Import required libraries:
import requests
import numpy as np
import pandas as pd

from google.colab import files

## Set-up urls for extracting data from [StatsBomb Github repo](https://github.com/statsbomb/open-data)

In [ ]:
scrape_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = scrape_url + "matches/{}/{}.json"
lineup_url = scrape_url + "lineups/{}.json"
event_url = scrape_url + "events/{}.json"

In [ ]:
comp_url.format(11, 42)

In [ ]:
scrape_url + "matches/" + str(43) + "/" + str(3) + ".json"

In [ ]:
event_url.format(15326)

## Competitions meta-data

In [ ]:
scrape_url

In [ ]:
# Get the relevant json file
competitions_json = requests.get(url=scrape_url + "competitions.json").json()

In [ ]:
type(competitions_json)

In [ ]:
competitions_json[0]

In [ ]:
for comp in competitions_json:
    print(comp["competition_name"], comp["season_name"])

In [ ]:
pd.json_normalize(competitions_json)

In [ ]:
sb_comp_meta_df = pd.json_normalize(competitions_json)

In [ ]:
# Save data to csv file on your local system
sb_comp_meta_df.to_csv('sb_competition_metadata.csv')
files.download('sb_competition_metadata.csv')

## La-Liga Match Meta data for all seasons

In [ ]:
sb_comp_meta_df

In [ ]:
comp_id = 43

In [ ]:
# Slice DF to obtain season ids for all seasons of La Liga:
season_ids = sb_comp_meta_df.loc[sb_comp_meta_df["competition_id"] == comp_id, "season_id"].to_list()

In [ ]:
sb_comp_meta_df.loc[sb_comp_meta_df["competition_id"] == comp_id, "season_id"]

In [ ]:
season_ids

In [ ]:
comp_url

In [ ]:
season_meta_data_df = pd.DataFrame()
for s_id in season_ids:
    season_json = requests.get(url=comp_url.format(comp_id, s_id)).json()
    df = pd.json_normalize(season_json)
    season_meta_data_df = season_meta_data_df.append(df)

In [ ]:
season_meta_data_df.columns

In [ ]:
season_meta_data_df["season.season_id"].unique()

In [ ]:
season_meta_data_df.columns

In [ ]:
season_meta_data_df.head()

In [ ]:
season_meta_data_df.groupby(["season.season_id"])["match_id"].count()

In [ ]:
# Save data to csv file on your local system
season_meta_data_df.to_csv('season_matchwise_data.csv')
files.download('season_matchwise_data.csv')

## Event data for one match in La-Liga

In [ ]:
match_id_list = season_meta_data_df["match_id"].unique()

In [ ]:
event_json = requests.get(url=event_url.format(match_id_list[0])).json()

In [ ]:
event_json

In [ ]:
pd.json_normalize(event_json)

In [ ]:
event_df = pd.json_normalize(event_json)

In [ ]:
event_df.columns.values

In [ ]:
event_df["type.name"].unique()

## Event data for La Liga 2019-20

In [ ]:
match_id_list = season_meta_data_df.loc[season_meta_data_df["season.season_id"] == 3, "match_id"].unique()

In [ ]:
match_id_list

In [ ]:
matchwise_events_df = pd.DataFrame()
for m_id in match_id_list:
    print("Working on", m_id)
    event_json = requests.get(url=event_url.format(m_id)).json()
    df = pd.json_normalize(event_json)
    df.insert(0, "match_id", m_id)
    matchwise_events_df = matchwise_events_df.append(df)

In [ ]:
matchwise_events_df.head()

In [ ]:
# Save data to csv file on your local system
matchwise_events_df.to_csv('matchwise_events_data.csv')
files.download('matchwise_events_data.csv')

In [ ]:
matchwise_events_df.shape